 # Exploración

 ## Imports

In [22]:
import os
import numpy as np
import pandas as pd
from IPython.display import display


 ## Lectura del fichero de datos

In [23]:
file = '.\\data\\utmb.1.csv'
df = pd.read_csv(file, low_memory=False)
pd.set_option('display.max_columns', df.shape[1])
# todo uncomment
# df[:3]


 ## Conversión de tipos de columnas

 #### Las columnas con la etiqueta crono están situadas a partir del índice 8. Las recuperamos directamente sin pasar por el mapa de columnas. Verificamos al final que podemos operar con los valores correctamente con una resta de tiempos cualquiera.

In [24]:
cronos = list(df)[8:]
for crono in cronos:
    print(crono)
    td = pd.to_timedelta(df[crono])
    df[crono] = td

df.Time[30] - df.Time[29]


Time
Timediff
ColDeVoza
LaCharme
Delevret
StGervais
Contamines
LaBalme
ContaR
Bellevue
LesHouches
GarePp
Bonhomme
Chapieux
ColSeigne
RefugeElisabetta
LacCombal
MtFavre
Checrouit
Courmayeur
Courmayeur2
Bertone
RifugioElena
Bonatti
Arnouvaz
ColFerret
LaPeule
LaFouly
PrazDeFort
Champex
Martigny
Bovinette
LaGiete
Trient
Tseppes
Catogne
Vallorcine
ColDesMontets
LaTeteAuxVents
LaFlegere
Argentiere
Gardes
1km
Arrivee


Timedelta('0 days 00:41:08')

 ## Verificaciones

 #### De entrada haremos algunas verificaciones básicas. Empezamos por comprobar la variable ```Arrivee```.

In [25]:
df.groupby('Year')['Arrivee'].min().sort_values()


Year
2012   10:32:36
2017   19:01:54
2014   20:11:44
2013   20:34:57
2011   20:36:43
2008   20:56:59
2006   21:06:06
2004   21:06:18
2015   21:09:15
2005   21:11:07
2007   21:31:58
2009   21:33:18
2016   22:00:02
2003        NaT
2010        NaT
Name: Arrivee, dtype: timedelta64[ns]

 #### Se ven tres valores anómalos
 1. 2012 El tiempo de llegada es la mitad de los demás años informados.
 2. 2003 No hay tiempo de llegada.
 3. 2010 No hay tiempo de llegada.

 #### Preparamos una función auxiliar para recuperar solamente los datos de las columnas con información, el variopinto esquema de cada año hace que en muchas variables no haya ningún dato; de esta manera podemos realizar el análisis con menos ruido.

In [26]:
# función auxiliar para recuperar un año  eliminando las
# columnas con todos los valores nans


def get_year(df, year):
    year_df = df.loc[df.Year == year]
    notnas = year_df.columns[year_df.notna().any()].tolist()
    return year_df[notnas]


 #### Año 2012

In [27]:
# todo uncomment
# get_year(df, 2012)[:3]


 #### Para 2012 vemos que los datos son correctos y parecen razonables, pero los puntos de paso y los tiempos indican que se trata de otra carrera. No podremos utilizar este año para contrastarlo con los demás. Como ni siquiera sabemos de qué carrera se trata, vamos a excluirlo directamente del resto del análisis. Hay que borrar las observaciones cuyo año es 2012 y, después, eliminar las variables exclusivas de ese año sabiendo que no tienen datos.

In [28]:
# excluimos las observaciones del año 2012 (es otra carrera)
df = df[df.Year != 2012]
# localizamos sus variables exclusivas sabiendo que serán nans
empties = df.columns[df.isna().all()].tolist()
display(empties)
# las borramos del dataframe
df.drop(columns=empties, inplace=True)
display(df.shape)


['ContaR', 'Bellevue', 'LesHouches', 'GarePp', '1km']

(30288, 47)

 #### Año 2003

In [29]:
# todo uncomment
# get_year(df, 2003)[:3]


 #### En 2003 se comprueba que no hay tiempos de paso intermedios, sino solamente el tiempo total. Sus observaciones pueden aportar información al análisis. Actualizaremos la columna ```Arrivee``` con el valor de ```Time```.

In [30]:
# mascara
# preparasmo una máscara
mask = df.Year == 2003
# actualizamos Arrivee
df.loc[mask, 'Arrivee'] = df.loc[mask, 'Time']
# todo uncomment
#get_year(df, 2003)[:3]


 #### Año 2010

In [31]:
get_year(df, 2010)[:3]


,Id,Year,Bib,Name,Rank,Category,Team,Nationality,Time,Delevret,StGervais
12876,0,2010,2216,CLEMENT Benoît,1,V1 H,,,0 days,01:53:45,02:52:44
12877,1,2010,3018,AGUETTAZ Christophe,2,V1 H,,,0 days,01:57:42,03:00:38
12878,2,2010,3913,PECH Philippe,3,V1 H,,,0 days,01:55:20,02:52:30


 #### 2010 no es un gran año para nuestra tarea de análisis, solo hay dos columnas de paso y no hay tiempo total. Mantendremos las observaciones porque pueden ser útiles para el análisis de participantes cuando no se requieran tiempos.

 #### Por último, examinamos todos los años para comprobar su calidad –al menos las de las primeras filas–.

In [32]:
years = np.arange(2003, 2018)
for y in years:
    if y != 2012:
        # todo del print
        print(y)
        # todo uncomment
        # display(get_year(df, y).head(3))


2003
2004
2005
2006
2007
2008
2009
2010
2011
2013
2014
2015
2016
2017


 #### New kid in town! Todo parece correcto excepto un tiempo de paso negativo en el año 2005 para el corredor con dorsal 1 en Contamines. Revisemos los tiempos negativos.

In [33]:
# recuperamos los datos de tiempos por posición
# sabiendo que son consecutivos
cronos = df[list(df)[8:]]

#función auxiliar reutilizable


def check_less_than_0(cronos):
    # filtramos tiempos negativos y no nans
    less_than_0 = (cronos < pd.Timedelta(0)) & pd.notna(cronos)
    # sumamos el resultado para contarlos (suma de booleanos)
    totals_lt0 = less_than_0.sum().copy()
    # guardamos los que tienen al menos un valor negativo ordenadamente
    totals_gt0 = totals_lt0[totals_lt0 > 0].sort_values(ascending=False)
    display(totals_gt0)
    return totals_gt0.index.values


cols_lt0 = check_less_than_0(cronos)


Courmayeur    41
Contamines     3
ColDeVoza      2
Tseppes        1
LaFouly        1
dtype: int64

 #### No son muchos, pero no es correcto dejarlos en el dataframe. Los ponemos a  ```nan```.

In [34]:
for col in cols_lt0:
    mask = (cronos[col] < pd.Timedelta(0)) & pd.notna(cronos[col])
    df.loc[mask, col] = np.nan

cronos = df[list(df)[8:]]
check_less_than_0(cronos)


Series([], dtype: int64)

array([], dtype=object)

 #### Vamos a ver ahora la calidad de la variable Nationality.

In [35]:
# pasamos todos los valores a minúsculas
df.Nationality = df.Nationality.str.lower()
# agrupamos, contamos y ordenamos los países
countries = df.groupby('Nationality')[
    'Nationality'].count().sort_values(ascending=False)
# mostramos los primeros
display(countries[:5])
# mostramos los últimos
display(countries[-5:])
# mostramos todos los valores
display(countries.index.values)


Nationality
      14601
fr     7938
es     1591
it     1116
gb      888
Name: Nationality, dtype: int64

Nationality
by    1
pa    1
cy    1
ne    1
bn    1
Name: Nationality, dtype: int64

array([' ', 'fr', 'es', 'it', 'gb', 'jp', 'de', 'ch', 'us', 'be', 'pl',
       'pt', 'cn', 'hk', 'au', 'at', 'se', 'hu', 'ca', 'nl', 'gr', 'ie',
       'ar', 'dk', 'cz', 'hr', 'ro', 'br', 'si', 'lv', 'fi', 'lu', 'sk',
       'sg', 'tr', 'no', 'bg', 'cl', 'nz', 'is', 'ru', 'mx', 'za', 'ph',
       'kr', 'co', 'my', 'tn', 'ec', 'tw', 'il', 'np', 've', 'in', 'ua',
       'gt', 'pe', 'lt', 'rs', 'id', 'ba', 'uy', 'ma', 'th', 'mk', 'lb',
       'py', 'cr', 'ir', 'dz', 'ad', 'ge', 'sv', 'sm', 'vn', 'ee', 'mt',
       'mu', 'bo', 'me', 'by', 'pa', 'cy', 'ne', 'bn'], dtype=object)

 #### **¡Cuidado!**, hay un país con un espacio en blanco como código. Lo tendremos en cuenta más adelante.

 #### Revisamos las categorías. En los listados de arriba se veían algunas categorías con espacios.

In [36]:
# pasamos todos los valores a minúsculas
df.Category = df.Category.str.lower()
# eliminamos los espacios
df.Category = df.Category.str.replace(' ', '')
# agrupamos, contamos y ordenamos los países
categories = df.groupby('Category')['Category'].count()
# mostramos los primeros
display(categories[:5])
# mostramos los últimos
display(categories[-5:])
# mostramos todos los valores
display(categories.index.values)


Category
esf       5
esh      79
juh       3
sef     947
seh    9645
Name: Category, dtype: int64

Category
v2h    5185
v3f      48
v3h     809
v4f       1
v4h      51
Name: Category, dtype: int64

array(['esf', 'esh', 'juh', 'sef', 'seh', 'v1f', 'v1h', 'v2f', 'v2h',
       'v3f', 'v3h', 'v4f', 'v4h'], dtype=object)

 #### La variable categoría está formada por la categoría y el sexo. Vamos a separarlas.

In [37]:
# creamos la columna sex cogiendo el ultimo carácter de category
df['Sex'] = df.Category.str[2]
display(df['Sex'][:3])
# suprimimos el sexo de categoría
df['Category'] = df.Category.str[:2]
# por conveniencia, reordenamos las columnas insertando el sexo en el índice 8
# para mantener los timedeltas juntos al final del dataframe
cols = list(df)
cols.remove('Sex')
cols.insert(8, 'Sex')
df = df[cols]


0    h
1    h
2    h
Name: Sex, dtype: object

 #### Guardamos los resultados en un nuevo csv para seguir la exploración en otro notebook.

In [38]:
file = '.\\data\\utmb.2.csv'
df.to_csv(file, index=False)
